<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/10_text_mining_with_finetuned_LMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストマイニングのための言語モデルのfinetuning

## 技術的な背景
* 今ではBERT largeの規模の言語モデルでも気軽に使える。
* 今までならトピックモデルを使っていただろうトピック抽出も気軽に行える。
* さらに、今ではBERT large規模の言語モデルのファインチューニングも気軽に実行できる。
* ということは、分析対象のテキスト集合を使って言語モデルをファインチューニングすれば・・・
* 分析対象のテキスト集合向けにカスタマイズされたトピック抽出も、気軽にできる。

## 準備

### インストール

In [ ]:
!pip install -U datasets captum fugashi sentencepiece unidic-lite

* transformersのバージョンが新しすぎると後でエラーが出る。

In [ ]:
!pip install transformers==4.45.2 sentence-transformers==3.1.1

### インポート

In [ ]:
from tqdm.auto import tqdm
import numpy as np

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn
from torch.nn.functional import normalize

from datasets import Dataset, DatasetDict, load_dataset
from transformers import set_seed
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.losses import TripletLoss
from sentence_transformers.evaluation import TripletEvaluator

set_seed(0)

## 言語モデルを使ったトピック抽出

### データセット
* ライブドアニュースコーパスを使う。
  * 分類カテゴリは今回は使わない。

In [ ]:
!wget https://github.com/tomonari-masada/course2024-nlp/raw/refs/heads/main/livedoor_ds.tar.gz
!tar zxf livedoor_ds.tar.gz

In [ ]:
from datasets import load_from_disk
ds = load_from_disk("livedoor_ds")

In [ ]:
ds

In [ ]:
category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

In [ ]:
num_categories = len(set(ds["train"]["category"]))
assert len(category_names) == num_categories

In [ ]:
ds["train"][0]

### 言語モデル
* 今回は`cl-nagoya/ruri-large`を使う。
  * https://zenn.dev/hpp/articles/b5132c64c40d24
  * https://github.com/sbintuitions/JMTEB/blob/main/leaderboard.md
* sentence transformersというライブラリを使ってモデルを操作する。
  * Hugging Faceのtransformersよりも初心者に優しい。

In [ ]:
model_id = "cl-nagoya/ruri-large"
model = SentenceTransformer(model_id)

In [ ]:
model

In [ ]:
list(model[0].modules())[1]

* すでにGPUに移動されている。

In [ ]:
model.device

### テキストの埋め込み

* 今回は（時間節約のため）タイトルを使う。

In [ ]:
embeddings = {}
for key in ds:
  embeddings[key] = model.encode(
      ds[key]["title"],
      show_progress_bar=True,
  )

### テキストのクラスタリング

* クラスタ数は適当に設定する。

In [ ]:
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=0)
kmeans.fit(embeddings["train"])
centers = kmeans.cluster_centers_

* クラスタの重心に近い順に数件のテキストを表示させる。

In [ ]:
similarities = cosine_similarity(embeddings["train"], centers)
for i in range(similarities.shape[-1]):
  indices = np.argsort(- similarities[:,i])[:5]
  for index in indices:
    print(f"{i:d} {index:d} " + ds["train"]["title"][index])

* クラスタリングの評価
  * 今回のデータセットでは正解ラベルを使った評価が可能。

In [ ]:
def evaluate_cluster(labels_true, labels_pred):
  return (
      metrics.adjusted_rand_score(labels_true, labels_pred),
      metrics.adjusted_mutual_info_score(labels_true, labels_pred),
      metrics.v_measure_score(labels_true, labels_pred)
  )

In [ ]:
rand_score, ami_score, v_measure = evaluate_cluster(ds["train"]["category"], kmeans.labels_)
print(f"adjusted Rand\t{rand_score:.3f}\nAMI score\t{ami_score:.3f}\nV-measure\t{v_measure:.3f}")

### プロンプトを使用した埋め込み

* 開発元はプロンプトの利用を推奨している。
  * https://zenn.dev/hpp/articles/b5132c64c40d24#fine-tuning

In [ ]:
embeddings = {}
for key in ds:
  embeddings[key] = model.encode(
      ds[key]["title"],
      show_progress_bar=True,
      prompt="クエリ: ",
  )

In [ ]:
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=0)
kmeans.fit(embeddings["train"])
centers = kmeans.cluster_centers_

In [ ]:
rand_score, ami_score, v_measure = evaluate_cluster(ds["train"]["category"], kmeans.labels_)
print(f"adjusted Rand\t{rand_score:.3f}\nAMI score\t{ami_score:.3f}\nV-measure\t{v_measure:.3f}")

### クラスタのラベリング

* 下に示すのは、あくまで一つの方法。

* 日本語テキストを形態素解析できるようにする。
  * セッションの再起動はおそらく不要。


In [ ]:
!python -m spacy download ja_core_news_sm

* 形態素解析をおこない、名詞と固有名詞だけを残す。

In [ ]:
label_pos_tags = ["NOUN", "PROPN"]

nlp = spacy.load("ja_core_news_sm")
corpus = {}
for key in ds:
  corpus[key] = []
  for text in tqdm(ds[key]["title"]):
    corpus[key].append(" ".join(
        [token.lemma_
         for token in nlp(text) if token.pos_ in label_pos_tags
        ]
    ))

* scikit-learnのTfidfVectorizerを使う。
  * min_dfやmax_dfの設定は、クラスタのラベリングに使う単語を絞り込む。

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.1, lowercase=False)
vectorizer.fit(corpus["train"])
vocab = np.array(vectorizer.get_feature_names_out())
X_train = vectorizer.transform(corpus["train"]).toarray()

In [ ]:
vocab

* 単語を含むテキストのtf-idf値を使って、テキストの重みづけ線形和を求める。
* 得られた線形和をその単語の埋め込みとみなす。

In [ ]:
vocab_embeddings = np.dot((X_train / X_train.sum(0)).T, embeddings["train"])

* テキストのクラスタの重心に近い順に20個の単語を表示させる。

In [ ]:
topic_words = []
similarities = cosine_similarity(vocab_embeddings, centers)
for i in range(similarities.shape[-1]):
  indices = np.argsort(- similarities[:,i])
  topic_words.append(f"{i:d} " + " ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))

## ファインチューニング

### トリプレットデータを作成する
* anchor, positive, negativeという3つのテキストの組。
  * positiveはanchorと同じクラスに属するテキスト。
  * negativeはanchorと異なるクラスに属するテキスト
* この三つ組のテキストを、後でファインチューニングに使う。

In [ ]:
triplet_dataset = {}
for key in ds:

  categorized = [list() for i in range(num_categories)]
  for example in ds[key]:
    categorized[example["category"]].append(example["title"])
  category_size = [len(categorized[i]) for i in range(num_categories)]

  anchors, positives, negatives = [], [], []
  for i in range(num_categories):
    indices = i + np.random.randint(1, num_categories, category_size[i])
    indices = indices % num_categories
    anchors += categorized[i]
    positives += [
        categorized[i][np.random.randint(0, category_size[i])]
        for _ in indices
    ]
    negatives += [
        categorized[j][np.random.randint(0, category_size[j])]
        for j in indices
    ]

  triplet_dataset[key] = Dataset.from_dict({
      "anchors": anchors,
      "positives": positives,
      "negatives": negatives,
  })

triplet_dataset = DatasetDict(triplet_dataset)

In [ ]:
train_dataset = triplet_dataset["train"]
eval_dataset = triplet_dataset["validation"]
test_dataset = triplet_dataset["test"]

In [ ]:
train_dataset[0]

### 損失関数

* トリプレット損失関数を使う。
  * anchorとpositiveの埋め込みベクトルは接近し・・・
  * anchorとnegativeの埋め込みベクトルは遠ざかるように・・・
  * ファインチューニングが進む。

In [ ]:
loss = TripletLoss(model)

### trainerの設定

In [ ]:
args = SentenceTransformerTrainingArguments(
    output_dir=f"models/{model_id}_livedoor-title-triplet",
    max_steps=300,
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    bf16=True,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
)

### evaluatorの設定

In [ ]:
dev_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchors"],
    positives=eval_dataset["positives"],
    negatives=eval_dataset["negatives"],
)

* ファインチューニングの実行前に、evaluatorに評価させてみる。

In [ ]:
dev_evaluator(model)

### trainerの作成

In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)

* trainableなパラメータの個数を確認してみる。
  * 100%になっているはず。

In [ ]:
def print_trainable_parameters(model):
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} "
      f"|| all params: {all_param} "
      f"|| trainable%: {100 * trainable_params / all_param}"
  )

In [ ]:
print_trainable_parameters(model)

In [ ]:
trainer.get_num_trainable_parameters()

### ファインチューニングの実行

In [ ]:
trainer.train()

### ファインチューニング後のモデルによるテキストの埋め込み

In [ ]:
embeddings = {}
for key in ds:
  embeddings[key] = model.encode(
      ds[key]["title"],
      show_progress_bar=True,
      prompt="クエリ: ",
  )

In [ ]:
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=0)
kmeans.fit(embeddings["train"])
centers = kmeans.cluster_centers_

In [ ]:
rand_score, ami_score, v_measure = evaluate_cluster(ds["train"]["category"], kmeans.labels_)
print(f"adjusted Rand\t{rand_score:.3f}\nAMI score\t{ami_score:.3f}\nV-measure\t{v_measure:.3f}")

### 語彙の埋め込み

In [ ]:
vocab_embeddings = np.dot((X_train / X_train.sum(0)).T, embeddings["train"])

### クラスタのラベリング

In [ ]:
topic_words = []
similarities = cosine_similarity(vocab_embeddings, centers)
for i in range(similarities.shape[-1]):
    indices = np.argsort(- similarities[:,i])
    topic_words.append(f"{i:d} " + " ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))

## 言語モデルの解釈
* Integrated Gradientsという手法を使う。
  * Captumというライブラリに実装がある。
* 特定のクラスタにテキストが所属する"根拠"を調査できる。
  * どのトークンがその所属に効いているかを可視化できる。

* モデルのパラメータを微分できない状態にする。
  * メモリを節約するため。

In [ ]:
for param in model.parameters():
  param.requires_grad = False

* モデルをCPUに移す。
  * 無料版のGoogle ColabではGPUのメモリが足りなくなるため。

In [ ]:
model.to("cpu").eval();

### 準備

In [ ]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

tokenizer = model.tokenizer
token_reference = TokenReferenceBase(reference_token_idx=tokenizer.pad_token_id)

### Captum向けの埋め込みの書き方

* input_idsとattention_maskを使った埋め込みに書き換える。
  * Captumではsentence transformersにおける言語モデルのencodeメソッドは使えない。

In [ ]:
text = "クエリ: " + ds["train"]["title"][0]
encodings = tokenizer(text, padding=True, return_tensors="pt")
encodings = encodings.to(model.device)
input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]
with torch.no_grad():
  embedding = model({"input_ids": input_ids, "attention_mask": attention_mask})["sentence_embedding"]
  st_embedding = model.encode(text)

* encodeメソッドと埋め込みベクトルが一致することを確認する。

In [ ]:
((embedding - st_embedding) ** 2).sum()

### テキストとクラスタ重心との類似度を求めるヘルパ関数

* 類似度はコサイン類似度で求める。
  * ベクトルの長さが1なので、大小関係はユークリッド距離と同じ。

In [ ]:
cos_sim = nn.CosineSimilarity(dim=-1)
cluster_centers = torch.tensor(centers, device=model.device)

def predict(input_ids, attention_mask):
  embedding = model({
      "input_ids": input_ids,
      "attention_mask": attention_mask,
  })["sentence_embedding"]
  return cos_sim(
      cluster_centers.unsqueeze(0),
      embedding.unsqueeze(1)
  )

* 試しに一つのテキストについて各クラスタとの類似度を計算してみる。

In [ ]:
text = "クエリ: " + ds["train"]["title"][0]
print(text)
encodings = tokenizer(text, padding=True, return_tensors="pt")
encodings.to(model.device)
similarities = predict(encodings.input_ids, encodings.attention_mask)
similarities

### テキストと特定のクラスタの重心との類似度を求めるヘルパ関数

In [ ]:
def cluster_similarity_forward_func(input_ids, attention_mask, cluster_id):
  similarities = predict(input_ids, attention_mask)
  return similarities[:,cluster_id]

In [ ]:
idx = 0

text = "クエリ: " + ds["train"]["title"][idx]
encodings = tokenizer(text, padding=True, return_tensors="pt")
encodings.to(model.device)
cluster_similarity_forward_func(
    encodings.input_ids,
    encodings.attention_mask,
    kmeans.labels_[idx],
)

### Integrated Gradientの初期化

* モデルのembedding layerを取得する必要がある。

In [ ]:
list(model[0].modules())[1]

In [ ]:
list(model[0].modules())[1].embeddings

In [ ]:
lig = LayerIntegratedGradients(
    cluster_similarity_forward_func,
    list(model[0].modules())[1].embeddings.word_embeddings,
)

### 可視化のためのヘルパ関数

* 可視化結果を蓄えるリスト

In [ ]:
vis_data_records_ig = []

* 可視化のためのヘルパ関数

In [ ]:
def add_attributions_to_visualizer(attributions, text, pred_prob, pred_class, true_class,
                                   attr_class, convergence_scores, vis_data_records):
  attributions = attributions.cpu()
  attributions = attributions.sum(dim=-1).squeeze(0)
  attributions = attributions / torch.norm(attributions)
  attributions = attributions.cpu().detach().numpy()
  vis_data_records.append(
      visualization.VisualizationDataRecord(
          attributions,
          pred_prob,
          pred_class,
          true_class,
          attr_class,
          attributions.sum(),
          text,
          convergence_scores,
      )
  )

### 解釈を実行する関数

In [ ]:
def interpret_text(text, attr_class=None, n_steps=50):
  encodings = tokenizer(text, padding=True, return_tensors="pt")
  encodings = encodings.to(model.device)
  input_ids = encodings.input_ids
  attention_mask = encodings.attention_mask
  tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
  reference_input_ids = token_reference.generate_reference(
      len(tokens),
      device=model.device,
  ).unsqueeze(0)

  similarities = predict(
      input_ids,
      attention_mask,
  )
  prediction = similarities.argmax().item()
  if attr_class is None:
    attr_class = prediction
  print(
      f"prediction={prediction} "
      f"cos_sim={similarities.max().item():.3f} ",
      end=""
  )

  attributions_ig, delta = lig.attribute(
      input_ids,
      reference_input_ids,
      additional_forward_args=(attention_mask, attr_class),
      n_steps=n_steps,
      return_convergence_delta=True,
  )
  print(f"convergence delta={delta.item():.3e} when n_steps={n_steps}")

  add_attributions_to_visualizer(
      attributions_ig,
      tokens,
      similarities.max().item(),
      str(prediction),
      str(prediction),
      str(attr_class),
      delta,
      vis_data_records_ig,
  )
  return prediction


### Integrated Gradientsの実行

In [ ]:
vis_data_records_ig = []
for n_steps in [50, 100]:
  interpret_text(ds["train"]["title"][0], n_steps=n_steps)

### 解釈結果の可視化

* ステップ数によって結果が微妙に変わることもある。

In [ ]:
visualization.visualize_text(vis_data_records_ig);

* 50件ぐらいのテキストのクラスタへの所属をまとめて解釈する。

In [ ]:
for i in range(50):
  example = ds["test"][i]
  print("="*80)
  print(category_names[example["category"]], end=" ")
  vis_data_records_ig = []
  prediction = interpret_text("クエリ: " + example["title"], n_steps=50)
  print("\t" + topic_words[prediction])
  visualization.visualize_text(vis_data_records_ig);